In [1]:
import sys
import os
import json
import pandas as pd
from IPython.display import IFrame

# Change dir to root dir
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
os.chdir(parent_dir)
print("Current Working Directory:", os.getcwd())

from src.data import generate_df, convert_to_gdf, generate_gdf, preprocess_df
from src.plot import plot_data, plot_kde, plot_correlation_matrix, plot_map
from src.model import light_gbm, predict_light_gbm_model
from src.data import generate_df, convert_to_gdf, generate_gdf, preprocess_df, preprocess_gdf, filter_nam_outside_vri, get_nam_outside_vri_nearest_station
from src.scripts.generateNamCSV import generate_nam_csv
from src.scripts.generateElevationCSV import generate_elevation_csv


Current Working Directory: /home/ssahar/private/DSC180B


In [2]:
with open('config/data_params.json') as fh:
        data_params = json.load(fh)    

raw_data_path = [os.path.join('./data/raw', file_path) for file_path in data_params["raw_data"]]
modified_data_path = [os.path.join('./data/modified', file_path) for file_path in data_params["modified_data"]]
output_model_path = [os.path.join('./data/modified', file_path) for file_path in data_params["model_prediction"]]

In [3]:
# Reading raw data
gis_weather_station, src_vri_snapshot, nam, windspeed_snapshot = generate_df(raw_data_path) 
gis_weather_station, windspeed_snapshot = preprocess_df(gis_weather_station, windspeed_snapshot)

# Reading filtered data with elevation from API
gis_weather_station_with_elevation, nam_with_elevation = generate_df(modified_data_path)
gis_weather_station_with_elevation_gpd, src_vri_snapshot_gpd, nam_with_elevation_gpd = generate_gdf(
    gis_weather_station_with_elevation, src_vri_snapshot, nam_with_elevation)

# Readiing data from LightGBM model
nam_within_vri_prediction, nam_outside_vri_prediction = generate_df(output_model_path)
nam_within_vri_prediction_gpd, nam_outside_vri_prediction_gpd = convert_to_gdf(nam_within_vri_prediction, True), convert_to_gdf(nam_outside_vri_prediction)

Weather Station CRS:    EPSG:4326
VRI Polygon CRS:        EPSG:4326
NAM CRS:                EPSG:4326


/home/ssahar/private/DSC180B/src/data.py:9: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  return [pd.read_csv(file) for file in input_list]


In [4]:
df = nam_within_vri_prediction_gpd.groupby(['geometry'])[['abs_wind_speed_error']].mean().reset_index()
df

,geometry,abs_wind_speed_error
0,POINT (-117.10672 32.54224),8.637497
1,POINT (-117.09059 32.54212),8.643125
2,POINT (-117.07446 32.54202),8.646338
3,POINT (-117.05774 32.5963),6.400400
4,POINT (-117.05789 32.5827),6.337770
...,...,...
957,POINT (-116.45962 32.63112),18.689873
958,POINT (-116.05371 32.72061),36.823705
959,POINT (-116.07048 32.69366),36.567835
960,POINT (-116.08725 32.66671),37.290560


In [5]:
plot_map(gis_weather_station_with_elevation_gpd, src_vri_snapshot_gpd, df, 'abs_wind_speed_error', "test_file.html")

In [6]:
IFrame("../plots/test_file.html", width=700, height=500)